In [2]:
# /////////////////////////////////////final code regional rural deck
import xlwings as xw

def copy_top_bottom_data(sheet_name, start_cell, sum_col_index, top_n, ascending, dest_sheet, dest_range):
    # File path
    file_path = r"C:\Users\thotnr\Desktop\powerbify25\Region Deck All Regionn updated.xlsx"
    
    # Open the workbook and sheets
    wb = xw.Book(file_path)
    src_sheet = wb.sheets[sheet_name]
    dest_sheet = wb.sheets[dest_sheet]
    
    # Identify the range starting from the provided start cell and expand to the table range
    data_range = src_sheet.range(start_cell).expand("table")
    
    # Get all data in the range (including headers)
    data = data_range.value
    
    # Sort the data by "Sum of Column" (ascending or descending based on the parameter)
    # Handle None by replacing it with a large or small number depending on sorting order
    sorted_data = sorted(
        data[1:], 
        key=lambda x: (float('-inf') if x[sum_col_index] is None else x[sum_col_index]), 
        reverse=not ascending
    )
    
    # Get the top N rows (excluding the header row)
    filtered_data = [data[0]] + sorted_data[:top_n]
    
    # Paste the filtered data into the destination sheet at the specified destination range
    dest_sheet.range(dest_range).value = filtered_data
    
    # Save the workbook (no need to close if you're still working with it)
    wb.save()
  
    print(f"Top {top_n if not ascending else 'Bottom'} {top_n} data copied from '{sheet_name}' to '{dest_sheet.name}' successfully.")

# Functions for specific tasks

# Top 10 ASD (copied to BW2)
def copy_top10_asd_data():
    copy_top_bottom_data(
        sheet_name="Top10 ASD", 
        start_cell="A6", 
        sum_col_index=0, 
        top_n=10, 
        ascending=False,  # Sort in descending order for top 10
        dest_sheet="Format", 
        dest_range="BW2"
    )

# Bottom 10 ASD (copied to BW17)
def copy_bottom10_asd_data():
    copy_top_bottom_data(
        sheet_name="Bottom10 ASD", 
        start_cell="A18", 
        sum_col_index=0, 
        top_n=10, 
        ascending=True,  # Sort in ascending order for bottom 10
        dest_sheet="Format", 
        dest_range="BW17"
    )

# Top 5 Dealer (copied to CM2)
def copy_top5_dealer_data():
    copy_top_bottom_data(
        sheet_name="TOP 5 Dealer", 
        start_cell="A766", 
        sum_col_index=0,  # Adjust column index if necessary
        top_n=5, 
        ascending=False,  # Sort in descending order for top 5
        dest_sheet="Format", 
        dest_range="CM2"
    )

# Bottom 5 Dealer (copied to CM17)
def copy_bottom5_dealer_data():
    copy_top_bottom_data(
        sheet_name="Bottom 5 Dealer", 
        start_cell="A4", 
        sum_col_index=0,  # Adjust column index if necessary
        top_n=5, 
        ascending=True,  # Sort in ascending order for bottom 5
        dest_sheet="Format", 
        dest_range="CM17"
    )

# Function to clear and paste data
def clear_and_paste(source_ws, source_start_cell, dest_ws, dest_start_cell, merge_ranges=None):
    # Unmerge destination cells before clearing contents
    dest_ws.range(dest_start_cell).expand().unmerge()

    # Clear all data in the destination before pasting
    dest_ws.range(dest_start_cell).expand().clear_contents()

    # Determine the range to copy based on start cell
    last_row = source_ws.range(source_start_cell).end('down').row
    last_col = source_ws.range(source_start_cell).end('right').column
    data_range = source_ws.range(source_start_cell, (last_row, last_col))

    # Paste the data
    dest_ws.range(dest_start_cell).value = data_range.value

    # Apply merging if specified
    if merge_ranges:
        for merge_range in merge_ranges:
            dest_ws.range(merge_range).merge()

# Function to copy and paste without "No match"
def copy_and_paste_without_no_match(source_ws, source_start_cell, dest_ws, dest_start_cell):
    last_row = source_ws.range(source_start_cell).end('down').row
    last_col = source_ws.range(source_start_cell).end('right').column
    data_range = source_ws.range(source_start_cell, (last_row, last_col)).value

    # Filter out rows containing "No match"
    filtered_data = [row for row in data_range if "No match" not in row]

    # Clear the destination sheet before pasting
    dest_ws.range(dest_start_cell).expand().clear_contents()

    # Paste the filtered data to the destination sheet
    dest_ws.range(dest_start_cell).value = filtered_data

# Main function to copy data to destination workbook
def copy_data(source_wb_path, dest_wb_path):
    # Open the source workbook
    source_wb = xw.Book(source_wb_path)

    # Open the destination workbook
    dest_wb = xw.Book(dest_wb_path)

    # 1. Format to Region
    clear_and_paste(source_wb.sheets['Format'], 'A3', dest_wb.sheets['Region'], 'A3', merge_ranges=[('A3:A4'), ('A5:A6'), ('A7:A8')])

    # 2. Format to Hub - Urban
    clear_and_paste(source_wb.sheets['Format'], 'A36', dest_wb.sheets['Hub - Urban'], 'A3', merge_ranges=[('A3:A4'), ('A5:A6'), ('A7:A8'), ('A9:A10'), ('A11:A12'), ('A13:A14'), ('A15:A16'), ('A17:A18')])

    # 3. Format to Hub - Rural
    clear_and_paste(source_wb.sheets['Format'], 'A151', dest_wb.sheets['Hub - Rural'], 'A3', merge_ranges=[('A3:A4'), ('A5:A6'), ('A7:A8'), ('A9:A10'), ('A11:A12'), ('A13:A14'), ('A15:A16'), ('A17:A18')])

    # 4. Top & Bottom 5 ASDs - Rural
    clear_and_paste(source_wb.sheets['Format'], 'BF3', dest_wb.sheets['Top & Bottom 5 ASDs - Rural'], 'A3')
    clear_and_paste(source_wb.sheets['Format'], 'BF18', dest_wb.sheets['Top & Bottom 5 ASDs - Rural'], 'A16')

    # 5. Top & Bottom 5 Dealers - Rural
    clear_and_paste(source_wb.sheets['Format'], 'AQ3', dest_wb.sheets['Top & Bottom 5 Dealers - Rural'], 'A3' )
    clear_and_paste(source_wb.sheets['Format'], 'AQ18', dest_wb.sheets['Top & Bottom 5 Dealers - Rural'], 'A12')

    # Additional logic for the new sheets without "No match" text:
    # P125 sheet logic
    copy_and_paste_without_no_match(source_wb.sheets['P125'], 'A2', dest_wb.sheets['Nil Retail ASDs - P125'], 'A2')

    # NS125 sheet logic
    copy_and_paste_without_no_match(source_wb.sheets['NS125'], 'A2', dest_wb.sheets['Nil Retail ASDs - NS125'], 'A2')

    # P150 sheet logic
    copy_and_paste_without_no_match(source_wb.sheets['P150'], 'A2', dest_wb.sheets['Nil Retail ASDs - P150'], 'A2')

    # N150N160 sheet logic
    copy_and_paste_without_no_match(source_wb.sheets['N150N160'], 'A2', dest_wb.sheets['Nil Retail ASDs - N150N160'], 'A2')

    # NS160NS200 sheet logic
    copy_and_paste_without_no_match(source_wb.sheets['NS160NS200'], 'A2', dest_wb.sheets['Nil Retail ASDs - NS160NS200'], 'A2')

    
 # Save and close the workbooks
    
    dest_wb.save()
    dest_wb.close()
    print("Data copied and pasted successfully!")

# Running the functions to copy top and bottom data
copy_top10_asd_data()
copy_bottom10_asd_data()
copy_top5_dealer_data()
copy_bottom5_dealer_data()

# Example usage for copying data to specific regional files
source_wb_path = r"C:\Users\thotnr\Desktop\powerbify25\Region Deck All Regionn updated.xlsx"

# Update for each region as needed
region_files = {
    # "AP": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _AP.xlsx",
    # "JH": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _JH.xlsx",
    # "BH": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _BH.xlsx",
    # "CG": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _CG.xlsx",
    # "DEL": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _DEL.xlsx",
    # "GJ": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _GJ.xlsx",
    # "HAR": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _HAR.xlsx",
    # "KAR": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _KAR.xlsx",
    # "KER": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _KER.xlsx",
    # "MAH": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _MAH.xlsx",
    # "MP": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _MP.xlsx",
    # "NE": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _NE.xlsx",
    # "ORI": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _ORI.xlsx",
    # "PB": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _PB.xlsx",
    # "RAJ": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _RAJ.xlsx",
    # "TG": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _TG.xlsx",
    # "TN": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _TN.xlsx",
    # "UPC": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _UPC.xlsx",
    # "UPE": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _UPE.xlsx",
    # "UPW": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _UPW.xlsx",
    "VID": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _VID.xlsx",
    # "WB": r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _WB.xlsx",
    # Add other regions here...
}

# Loop through the region files
for region, dest_wb_path in region_files.items():
    copy_data(source_wb_path, dest_wb_path)




Top 10 10 data copied from 'Top10 ASD' to 'Format' successfully.
Top Bottom 10 data copied from 'Bottom10 ASD' to 'Format' successfully.
Top 5 5 data copied from 'TOP 5 Dealer' to 'Format' successfully.
Top Bottom 5 data copied from 'Bottom 5 Dealer' to 'Format' successfully.
Data copied and pasted successfully!


In [1]:
import os
import math
import pandas as pd
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.enum.text import MSO_ANCHOR
from pptx.util import Pt  # Import to set font size

def load_data_from_excel(file_path, sheet_name, usecols, skiprows, nrows):
    """Load data from an Excel file."""
    try:
        return pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=skiprows, nrows=nrows)
    except Exception as e:
        print(f"Error loading data from {sheet_name}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

def format_dataframe(df, percentage_columns):
    """Format DataFrame columns as percentages or rounded integers."""
    for col in df.columns:
        if col in df.columns[percentage_columns]:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].apply(lambda x: f"{x * 100:.1f}%" if pd.notnull(x) else "")
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Round values to the nearest integer (e.g., 65.4 -> 65, 65.7 -> 66)
            df[col] = df[col].fillna(0).apply(lambda x: round(x)).astype(int).astype(str)
    return df








def fill_table_from_df(table, df, start_row, start_col, slide_index=None):
    """Fill PowerPoint table with data from DataFrame, ensuring dimensions match."""
    max_rows, max_cols = len(table.rows), len(table.columns)
    
    for i in range(len(df)):
        for j in range(len(df.columns)):
            if start_row + i >= max_rows or start_col + j >= max_cols:
                print(f"Skipping row {start_row + i}, column {start_col + j} - out of table bounds.")
                continue
            
            cell = table.cell(start_row + i, start_col + j)
            cell.text = str(df.iloc[i, j])
            align_cell_text(cell)
            
            # Apply font size 12 for specific slides
            if slide_index in [8, 9]:  # Check for slide 8 or 9
                for paragraph in cell.text_frame.paragraphs:
                    for run in paragraph.runs:
                        run.font.size = Pt(12)  # Set font size to 12 points
            
            # Highlight negative values in the 9th column (index 8), except for slide 8
            if j == 8 and slide_index not in [8, 16, 7]:  # Check if column is the 9th column and slide is not 8
                highlight_negative_value(cell, df.iloc[i, j])

            # Clear hyperlink from the cell if any (Currently, there's no direct method to do this)
            # You may need to manage hyperlinks externally based on your data

def align_cell_text(cell):
    """Center align the text in a cell."""
    for paragraph in cell.text_frame.paragraphs:
        paragraph.alignment = PP_ALIGN.CENTER
    cell.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

def highlight_negative_value(cell, value):
    """Highlight the cell if the value is negative."""
    try:
        numeric_value = float(value.replace('%', '').strip())
        if numeric_value < 0:
            cell.fill.solid()
            cell.fill.fore_color.rgb = RGBColor(255, 204, 204)  # Light red for negative values
        else:
            cell.fill.solid()
            cell.fill.fore_color.rgb = RGBColor(255, 255, 255)  # Set to white or default color
    except ValueError:
        pass  # Ignore if the value isn't numeric

def process_files_in_folder(excel_folder, ppt_folder):
    """Process each Excel file in the folder and save data to corresponding PowerPoint files."""
    for excel_filename in os.listdir(excel_folder):
        if excel_filename.endswith(".xlsx"):
            excel_file_path = os.path.join(excel_folder, excel_filename)
            
            # Derive the corresponding PowerPoint filename and path
            ppt_filename = f"Vistaar_MC Performance Report {excel_filename.split('_')[1].split('.')[0]}.pptx"
            ppt_file_path = os.path.join(ppt_folder, ppt_filename)
            
            # Skip if the corresponding PowerPoint file does not exist
            if not os.path.exists(ppt_file_path):
                print(f"PowerPoint file not found for {excel_filename}. Skipping...")
                continue
            
            presentation = Presentation(ppt_file_path)

            # Define sheet-to-slide mappings and data requirements
            sheets_to_process = [
                {'sheet_name': 'Region', 'slide_index': 3, 'start_row': 2, 'start_col': 0, 'usecols': 'A:L', 'skiprows': 1, 'nrows': 6, 'percentage_columns': [4, 7, 8, 9, 10, 11]},
                {'sheet_name': 'Hub - Urban', 'slide_index': 4, 'start_row': 2, 'start_col': 0, 'usecols': 'A:L', 'skiprows': 1, 'nrows': 16, 'percentage_columns': [4, 7, 8, 9, 10, 11]},
                {'sheet_name': 'Hub - Rural', 'slide_index': 5, 'start_row': 2, 'start_col': 0, 'usecols': 'A:L', 'skiprows': 1, 'nrows': 16, 'percentage_columns': [4, 7, 8, 9, 10, 11]},
                {'sheet_name': 'Top & Bottom 5 ASDs - Rural', 'slide_index': 8, 'start_row': 2, 'start_col': 0, 'usecols': 'A:N', 'skiprows': 1, 'nrows': 10, 'percentage_columns': [5, 8, 9, 10, 11, 12, 13]},
                {'sheet_name': 'Top & Bottom 5 ASDs - Rural', 'slide_index': 9, 'start_row': 2, 'start_col': 0, 'usecols': 'A:N', 'skiprows': 14, 'nrows': 10, 'percentage_columns': [5, 8, 9, 10, 11, 12, 13]}
            ]

            # Process each sheet and fill corresponding slide tables
            for sheet_info in sheets_to_process:
                df = load_data_from_excel(excel_file_path, sheet_info['sheet_name'], sheet_info['usecols'], sheet_info['skiprows'], sheet_info['nrows'])
                if df.empty:
                    continue  # Skip to the next sheet if there's no data
                
                df = format_dataframe(df, sheet_info['percentage_columns'])
                
                slide = presentation.slides[sheet_info['slide_index']]
                table = next((shape.table for shape in slide.shapes if shape.has_table), None)
                
                if table is not None:
                    fill_table_from_df(table, df, sheet_info['start_row'], sheet_info['start_col'], sheet_info['slide_index'])
                else:
                    print(f"No table found on slide {sheet_info['slide_index'] + 1} for {excel_filename}.")

            # Save the modified PowerPoint file in the same folder
            presentation.save(ppt_file_path)  # Save directly in the ppt_folder
            print(f"Data copied to {ppt_filename} successfully.")

# Define paths
excel_folder = r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region"
ppt_folder = r"C:\Users\thotnr\Desktop\adjustment"

# Process all files in the folder
process_files_in_folder(excel_folder, ppt_folder)


Data copied to Vistaar_MC Performance Report AP.pptx successfully.
Data copied to Vistaar_MC Performance Report BH.pptx successfully.
Data copied to Vistaar_MC Performance Report CG.pptx successfully.
Data copied to Vistaar_MC Performance Report DEL.pptx successfully.
Data copied to Vistaar_MC Performance Report GJ.pptx successfully.
Data copied to Vistaar_MC Performance Report HAR.pptx successfully.
Data copied to Vistaar_MC Performance Report JH.pptx successfully.
Data copied to Vistaar_MC Performance Report KAR.pptx successfully.
Data copied to Vistaar_MC Performance Report KER.pptx successfully.
Data copied to Vistaar_MC Performance Report MAH.pptx successfully.
Data copied to Vistaar_MC Performance Report MP.pptx successfully.
Data copied to Vistaar_MC Performance Report NE.pptx successfully.
Data copied to Vistaar_MC Performance Report ORI.pptx successfully.
Data copied to Vistaar_MC Performance Report PB.pptx successfully.
Data copied to Vistaar_MC Performance Report RAJ.pptx su

In [2]:
import xlwings as xw
from pptx import Presentation
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN

# File paths
excel_path = r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region\Region Deck _NE.xlsx"
powerpoint_path = r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck Power point\Vistaar_MC Performance Report NE.pptx"

# Excel sheet and ranges
sheet_name = "Top & Bottom 5 Dealers - Rural"
range1 = "A3:M7"
range2 = "A12:M16"

# Open Excel workbook
wb = xw.Book(excel_path)
sheet = wb.sheets[sheet_name]

# Read data from ranges
data1 = sheet.range(range1).value
data2 = sheet.range(range2).value

# Open PowerPoint presentation
prs = Presentation(powerpoint_path)
slide = prs.slides[7]  # Slide number 8 (index starts from 0)

# Define helper function to update table cells in PowerPoint
def update_table(table, data, start_row=2, percentage_columns=None, whole_number_columns=None):
    """Insert data starting from the specified row in the PowerPoint table."""
    for i, row in enumerate(data):
        for j, cell_value in enumerate(row):
            cell = table.cell(start_row + i, j)  # Start pasting from `start_row`
            # Format as whole number if in whole_number_columns
            if whole_number_columns and j in whole_number_columns:
                cell.text = str(int(cell_value)) if isinstance(cell_value, (int, float)) else str(cell_value)
            # Format as percentage if in percentage_columns
            elif percentage_columns and j in percentage_columns:
                cell.text = f"{cell_value:.2%}" if isinstance(cell_value, (int, float)) else str(cell_value)
            else:
                cell.text = str(cell_value)
                
            # Optional: Set font size and alignment
            for paragraph in cell.text_frame.paragraphs:
                paragraph.font.size = Pt(10)  # Adjust font size as needed
                paragraph.alignment = PP_ALIGN.CENTER

# Define the columns that should be formatted as percentages and whole numbers
percentage_columns1 = [4, 7, 8, 9, 10, 11, 12]  # Adjust indices as needed for data1
percentage_columns2 = [4, 7, 8, 9, 10, 11, 12]  # Adjust indices as needed for data2
whole_number_columns1 = [2, 3, 5, 6]  # Whole numbers for data1
whole_number_columns2 = [2, 3, 5, 6]  # Whole numbers for data2

# Find and update tables on the slide
table_shapes = [shape for shape in slide.shapes if shape.has_table]
if len(table_shapes) >= 2:
    table1 = table_shapes[0].table  # First table
    table2 = table_shapes[1].table  # Second table
    
    # Update PowerPoint tables with data from Excel
    update_table(table1, data1, start_row=2, percentage_columns=percentage_columns1, whole_number_columns=whole_number_columns1)
    update_table(table2, data2, start_row=2, percentage_columns=percentage_columns2, whole_number_columns=whole_number_columns2)

    # Save the PowerPoint file
    prs.save(powerpoint_path)
    print("Data copied from Excel to PowerPoint successfully.")
else:
    print("Error: Expected at least two tables on slide 8.")

# Close the Excel workbook
wb.close()


Data copied from Excel to PowerPoint successfully.


In [2]:
import os
import xlwings as xw
from pptx import Presentation
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN

# Define folder paths
excel_folder_path = r"C:\Users\thotnr\Desktop\powerbify25\Regional Deck All Region"
powerpoint_folder_path = r"C:\Users\thotnr\Desktop\adjustment"

# Excel sheet and ranges
sheet_name = "Top & Bottom 5 Dealers - Rural"
range1 = "A3:M7"
range2 = "A12:M16"

# Define columns for formatting
percentage_columns = [4, 7, 8, 9, 10, 11, 12]  # Indices for percentage columns
whole_number_columns = [2, 3, 5, 6]  # Indices for whole number columns

# Helper function to update table cells in PowerPoint
def update_table(table, data, start_row=2):
    """Insert data starting from the specified row in the PowerPoint table."""
    for i, row in enumerate(data):
        for j, cell_value in enumerate(row):
            cell = table.cell(start_row + i, j)
            # Format as whole number if in whole_number_columns
            if j in whole_number_columns:
                cell.text = str(round(cell_value)) if isinstance(cell_value, (int, float)) else str(cell_value)
            # Format as percentage if in percentage_columns
            elif j in percentage_columns:
                cell.text = f"{cell_value:.2%}" if isinstance(cell_value, (int, float)) else str(cell_value)
            else:
                cell.text = str(cell_value)

            # Set font size and alignment
            for paragraph in cell.text_frame.paragraphs:
                paragraph.font.size = Pt(14)
                paragraph.alignment = PP_ALIGN.CENTER

# Iterate over each Excel file in the Excel folder
for excel_filename in os.listdir(excel_folder_path):
    if excel_filename.endswith(".xlsx"):
        excel_path = os.path.join(excel_folder_path, excel_filename)
        
        # Open Excel workbook and read data
        wb = xw.Book(excel_path)
        sheet = wb.sheets[sheet_name]
        data1 = sheet.range(range1).value
        data2 = sheet.range(range2).value

        # Find a matching PowerPoint file
        ppt_filename = f"Vistaar_MC Performance Report {excel_filename.split('_')[1].split('.')[0]}.pptx"
        powerpoint_path = os.path.join(powerpoint_folder_path, ppt_filename)
        
        if os.path.exists(powerpoint_path):
            # Open PowerPoint presentation
            prs = Presentation(powerpoint_path)
            slide = prs.slides[7]  # Slide number 8 (index starts from 0)

            # Find tables on the slide
            table_shapes = [shape for shape in slide.shapes if shape.has_table]
            if len(table_shapes) >= 2:
                table1 = table_shapes[0].table  # First table
                table2 = table_shapes[1].table  # Second table

                # Update tables with data from Excel
                update_table(table1, data1, start_row=2)
                update_table(table2, data2, start_row=2)

                # Save changes to PowerPoint file
                prs.save(powerpoint_path)
                print(f"Data copied from {excel_filename} to {ppt_filename} successfully.")
            else:
                print(f"Error: Expected at least two tables on slide 8 in {ppt_filename}.")
            
            # Close the PowerPoint presentation
            prs = None

        else:
            print(f"No matching PowerPoint file found for {excel_filename}.")

        # Close the Excel workbook
        wb.save()
        wb.close()


Data copied from Region Deck _AP.xlsx to Vistaar_MC Performance Report AP.pptx successfully.
Data copied from Region Deck _BH.xlsx to Vistaar_MC Performance Report BH.pptx successfully.
Data copied from Region Deck _CG.xlsx to Vistaar_MC Performance Report CG.pptx successfully.
Data copied from Region Deck _DEL.xlsx to Vistaar_MC Performance Report DEL.pptx successfully.
Data copied from Region Deck _GJ.xlsx to Vistaar_MC Performance Report GJ.pptx successfully.
Data copied from Region Deck _HAR.xlsx to Vistaar_MC Performance Report HAR.pptx successfully.
Data copied from Region Deck _JH.xlsx to Vistaar_MC Performance Report JH.pptx successfully.
Data copied from Region Deck _KAR.xlsx to Vistaar_MC Performance Report KAR.pptx successfully.
Data copied from Region Deck _KER.xlsx to Vistaar_MC Performance Report KER.pptx successfully.
Data copied from Region Deck _MAH.xlsx to Vistaar_MC Performance Report MAH.pptx successfully.
Data copied from Region Deck _MP.xlsx to Vistaar_MC Performa